# Testing functionality of the AddPathHook function

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup for Postgres and queries from the stats workload

In [44]:
import sys, os
import postbound as pb
from postbound.optimizer import presets


# Setup instance and load workload
postgres_instance = pb.postgres.connect(connect_string="dbname=stats user=postbound host=localhost")
presets.apply_standard_system_options()
stats_workload = pb.workloads.stats()

In [45]:
# Select query from the workload
query_1 = stats_workload["q-1"]
query_2 = stats_workload["q-2"]
query_30 = stats_workload["q-30"]

### Setup of the Textbook Optimization Pipeline
* Using the Textbook Pipeline to enable the PostgresDynProg Enumerator/Planner
* optimizing query through pipeline 
* setup add_path_hook variant within instance of PostgresDynProg
* optimize queries

In [46]:
from postbound.optimizer.strategies.dynprog import PostgresDynProg
from postbound.optimizer.strategies.dynprog import AddPathHook
from add_path_hook import postgres_add_path_hook_standard

# Create the optimization pipeline
optimization_pipeline = pb.TextBookOptimizationPipeline(postgres_instance)
# Create a dynamic programming optimizer instance with wanted add_path hook
dynprog = PostgresDynProg(
    add_path_hook= postgres_add_path_hook_standard,
    target_db=postgres_instance)
optimization_pipeline.setup_plan_enumerator(dynprog)
optimization_pipeline.build()
optimization_pipeline.describe()


#optimize queries

optimized_query_1 = optimization_pipeline.optimize_query(query_1)
optimized_query_2 = optimization_pipeline.optimize_query(query_2)
optimized_query_30 = optimization_pipeline.optimize_query(query_30)
print(optimized_query_1)
print(optimized_query_2)
print(optimized_query_30)


/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path IndexScan(u)
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path BitmapScan(u)
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/native.py:226: UserWarning: Could not estimate the cost of memoize plan Memoize(SequentialScan(u)). Returning infinite costs.
  warnings.warn(f"Could not estimate the cost of memoize plan {plan}. Returning infinite costs.")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path NestedLoopJoin(SequentialScan(b), Memoize(SequentialScan(u)))
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimiz

/*=pg_lab=
  JoinOrder((b u))
  
  SeqScan(b)
  SeqScan(u)
  
  HashJoin(b u)
  
  
  Card(b u #79851)
  Card(b #79851)
  Card(u #40325)
*/
SELECT COUNT(*) FROM badges AS b, users AS u WHERE b.userid = u.id AND u.upvotes >= 0;
/*=pg_lab=
  JoinOrder((c b))
  
  SeqScan(c)
  SeqScan(b)
  
  HashJoin(c b)
  
  
  Card(c b #11091813)
  Card(c #134866)
  Card(b #79642)
*/
SELECT COUNT(*) FROM comments AS c, badges AS b WHERE c.userid = b.userid AND c.score = 0 AND b.date <= CAST('2014-09-11 14:33:06' AS timestamp);
/*=pg_lab=
  JoinOrder((v (p (b u))))
  
  IdxScan(v)
  SeqScan(p)
  SeqScan(b)
  IdxScan(u)
  
  HashJoin(p b u v)
  HashJoin(p b u)
  HashJoin(b u)
  
  
  Card(p b u v #603650)
  Card(p b u #74200)
  Card(p #37471)
  Card(b u #79851)
  Card(b #79851)
*/
SELECT COUNT(*) FROM votes AS v, posts AS p, badges AS b, users AS u WHERE u.id = v.userid AND u.id = p.owneruserid AND u.id = b.userid AND p.score >= 0 AND p.score <= 30 AND p.commentcount = 0 AND p.creationdate >= CAST('2010

/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path NestedLoopJoin(HashJoin(BitmapScan(v), SequentialScan(b)), Memoize(SequentialScan(p)))
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path NestedLoopJoin(SequentialScan(p), Memoize(HashJoin(BitmapScan(v), SequentialScan(b))))
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/dynprog.py:776: UserWarning: Rejecting illegal path NestedLoopJoin(HashJoin(HashJoin(SequentialScan(p), BitmapScan(u)), BitmapScan(v)), Memoize(SequentialScan(b)))
  warnings.warn(f"Rejecting illegal path {path}")
/postbound/pb-venv/lib/python3.12/site-packages/postbound/optimizer/strategies/native.py:226: UserWarning: Could not estimate the cost of memoize plan Memoize(HashJoin(Has

### Test out variant involving pruning before actually calling method

In [ ]:
from add_path_hook import postgres_add_path_hook_1
dynprog_2 = PostgresDynProg(
    add_path_hook= postgres_add_path_hook_1,
    target_db=postgres_instance
)

optimized_query_2_pruned = optimization_pipeline.query_execution_plan(query_2)
print(optimized_query_2_pruned)
